In [1]:
from dotenv import dotenv_values

# specify the name of the .env file name 
env_name = "../../llm.env" # change to your own .env file name
config = dotenv_values(env_name)

In [4]:
df = pd.read_csv('AnalyzedPDF/ChunksEmbedding.csv')

In [68]:
df[df['Quarter']==2]

,Id,Ticker,Year,Quarter,Chunk,PageNumber,LineNumber,Embedding
113,114,MSFT,23,2,Microsoft FY23 Second Quarter Earnings Confere...,1,1,"[-0.022043932229280472, -0.023832328617572784,..."
114,115,MSFT,23,2,"On the Microsoft Investor Relations website, y...",1,9,"[-0.023697681725025177, -0.005627374164760113,..."
115,116,MSFT,23,2,GAAP. They are included as additional clarifyi...,1,17,"[-0.012550131417810917, -0.0020706052891910076..."
116,117,MSFT,23,2,"same in constant currency, we will refer to th...",1,25,"[-0.017685849219560623, -0.02943631075322628, ..."
117,118,MSFT,23,2,"predictions, projections, or other statements ...",2,6,"[-0.00904887169599533, -0.01967030204832554, -..."
...,...,...,...,...,...,...,...,...
216,217,MSFT,23,2,"BRETT IVERSEN: Thanks, Brad. Joe, we have time...",31,13,"[0.00118646037299186, -0.040140919387340546, 0..."
217,218,MSFT,23,2,the coming quarters? Thank you. SATYA NADELLA:...,31,21,"[0.009347211569547653, -0.010082229971885681, ..."
218,219,MSFT,23,2,"going to be an Al app. That's, I think, the be...",32,1,"[0.005495850928127766, -0.0035756349097937346,..."
219,220,MSFT,23,2,"Sometimes, you will have ISVs who are differen...",32,9,"[-0.0043395268730819225, -0.02853129617869854,..."


In [82]:
df['Chunk'].iloc[11]

"Cosmos DB now supports PostgreSQL, making Azure the first cloud provider to offer a database service that supports both relational and NoSQL workloads. And, in Al, we are turning the world's most advanced models into platforms for customers. Earlier this month, we brought the power of Dall-E to Azure OpenAI service, helping customers like Mattel apply the breakthrough image generation model to commercial use cases for the first time. And Azure Machine Learning provides industry leading MLOps, helping organizations like 3M deploy, manage, and govern models. "

#### Prompt Template
##### Write a Prompt Tempalte. The prompt template should include all filter keys, so they can be referenced and input.

In [79]:
template = """
        You are a question answer generator.You are given two chunks of text, a ticker e.g. MSFT, Quarter, Year, as input. You will generate 10 relevant insightful questions and answers grounded in the text. The question should be such that information relevant to answer is available in both the chunks.
               
        An example output for this example is: 

        Question: For {ticker} FY{year} Q{quarter}, what is the growth rate?
        Answer: example answer goes here

        Based on ticker, quarter, year, the question can be phrased in different ways e.g. MSFT FY23 Q1, MSFT FY2023 1st quarter, e.t.c.
        In case the text question is not relevant, please skip the question and answer pair.
        input_text1: 
        {chunk_text1}
        input_text2:
        {chunk_text2}
        ticker: {ticker}
        quarter: {quarter}
        year: {year}
        """

#### Randomly pick filter parameters:

Add Filter Parameters to randomly pick them for extracting context (chunks). This will help diversify generating questions and answers. 
In the MSFT Financial Transcripts use-case, the Ticker name is MSFT, but you can easily add other ticker labels for a larger Financial dataset. `Year`,`Quarter`, and `Id` are the key parameters used in this use-case. In this notebook, we are using two random chunks (chunk ids) from specified filter keys (year, quarter).

###### TODO: Add Tools (an updated version of function calls) once it is available for working with newer models. Function Calls are currently deprecated for gpt models with versions beyond 07-01-2023.

In [70]:
Ticker = np.random.choice(df['Ticker'].unique())
Year = np.random.choice(df[df['Ticker']==Ticker]['Year'].unique())
Quarter = np.random.choice(df[(df['Ticker']==Ticker) & (df['Year']==Year)]['Quarter'].unique())
Id = np.random.choice(df[(df['Quarter']==Quarter) & (df['Year']==Year)]['Id'].unique())
Id2 = np.random.choice(df[(df['Quarter']==Quarter) & (df['Year']==Year) & (df['Id']!=Id)]['Id'].unique())

In [71]:
Ticker

'MSFT'

In [80]:
import os
from openai import AzureOpenAI

client = AzureOpenAI(
  api_key = config["OPENAI_API_KEY"],  
  api_version = config["OPENAI_API_VERSION"],
  azure_endpoint = config["OPENAI_API_BASE"]
)

response = client.chat.completions.create(
    model="gpt-35-turbo", # model = "deployment_name".
    messages=[
        {"role": "system", "content":"You are a generator of questions and answers for the given text." },
        {"role": "user", "content": template.format(chunk_text1=df['Chunk'].iloc[Id], chunk_text2=df['Chunk'].iloc[Id2], 
                                                    ticker=Ticker, year=str(Year), quarter=str(Quarter))}
    ]
)

#print(response)
# print(response.model_dump_json(indent=2))
print(response.choices[0].message.content)

Question 1: How many consumer subscribers did MSFT have, and what was the YoY growth, in the quarter 2 of fiscal year 2023?
Answer: MSFT had more than 63 million consumer subscribers in Q2 FY23, which is a 12 percent YoY growth.

Question 2: What new offering did MSFT introduce in Q2 FY23?
Answer: MSFT introduced Microsoft 365 "Basic" in Q2 FY23, bringing its premium offerings to more people.

Question 3: What is the total number of monthly active users for Teams in Q2 FY23?
Answer: Teams surpassed 280 million monthly active users in Q2 FY23.

Question 4: What is MSFT's momentum like for Teams since the pandemic?
Answer: MSFT's momentum for Teams has shown durability since the pandemic, with the platform taking share across every category, from collaboration, to chat, to meetings, to calling.

Question 5: Which third-party apps had each surpassed half a million active users in Q2 FY23?
Answer: The apps from Adobe, Atlassian, Polly, ServiceNow, and Workday had each surpassed half a mill

In [88]:
df[(df['Quarter']==Quarter) & (df['Year']==Year) & (df['Id']==Id)].Chunk.values[0]

"going to be an Al app. That's, I think, the best way to think about this transformation. On your characterization of the large customers, whether there is any changes in that trajectory, I would point back to, I think, some of the common theme around every large customer is looking to optimize the workload that they have at scale today, and plow some of that money back that they save into new project structure. That's sort of what the classic pattern of large customers is. Sometimes, you will have ISVs who are different, right? If an ISV is optimizing, they optimize and say, what is the new set of features to add at "

In [89]:
import re
_pattern = re.compile(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s')
retrieved_sentences=_pattern.split(df[(df['Quarter']==Quarter) & (df['Year']==Year) & (df['Id']==Id)].Chunk.values[0])
retrieved_sentences

['going to be an Al app.',
 "That's, I think, the best way to think about this transformation.",
 'On your characterization of the large customers, whether there is any changes in that trajectory, I would point back to, I think, some of the common theme around every large customer is looking to optimize the workload that they have at scale today, and plow some of that money back that they save into new project structure.',
 "That's sort of what the classic pattern of large customers is.",
 'Sometimes, you will have ISVs who are different, right?',
 'If an ISV is optimizing, they optimize and say, what is the new set of features to add at ']

In [31]:
response.choices[0].message.content

'Question: What does Microsoft Intelligent Data Platform offer to its customers?\nAnswer: Microsoft Intelligent Data Platform offers a complete data fabric that helps customers cut "integration tax" associated with bringing together siloed solutions.\n\nQuestion: Which company has standardized on Microsoft\'s data stack?\nAnswer: Mercedes-Benz has standardized on Microsoft\'s data stack to process and govern massive amounts of data.\n\nQuestion: What is Cosmos DB powering?\nAnswer: Cosmos DB is powering the world\'s most-demanding workloads at limitless scale.\n\nQuestion: What makes Azure unique among cloud providers in terms of database service?\nAnswer: Azure is the first cloud provider to offer a database service that supports both relational and non-relational data models, with Cosmos DB now supporting PostgreSQL.'